In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from os import listdir
import pandas as pd
import numpy as np
import glob
import cv2
import json
from os.path import expanduser
import splitfolders
import shutil
from define_path import Def_Path

from tqdm import tqdm

import torch 
import torchvision
from torchvision import models
from torchvision.models.detection.rpn import AnchorGenerator
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torchvision.transforms as T
from torchvision.transforms import functional as F
from torchsummary import summary

from sklearn.model_selection import train_test_split

import albumentations as A # Library for augmentations

import matplotlib.pyplot as plt 
from PIL import Image

import transforms, utils, engine, train
from utils import collate_fn
from engine import train_one_epoch, evaluate

t = torch.cuda.get_device_properties(0).total_memory
print(t)
torch.cuda.empty_cache()

r = torch.cuda.memory_reserved(0)
print(r)
a = torch.cuda.memory_allocated(0)
print(a)
# f = r-a  # free inside reserved

weights_path = '/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_sim_b1_e25_v0.pth'

16908615680
0
0


In [2]:
# to generalize home directory. User can change their parent path without entering their home directory
path = Def_Path()

parent_path =  path.home + "/Pictures/" + "Data/"

root_dir = parent_path + path.year + "-" + path.month + "-" + path.day + "/"

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.set_per_process_memory_fraction(0.9, 0)
print(device)

cuda


In [4]:
def train_transform():
    return A.Compose([
        A.Sequential([
            A.RandomRotate90(p=1), # Random rotation of an image by 90 degrees zero or more times
            A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, brightness_by_max=True, always_apply=False, p=1), # Random change of brightness & contrast
        ], p=1)
    ],
    keypoint_params=A.KeypointParams(format='xy'), # More about keypoint formats used in albumentations library read at https://albumentations.ai/docs/getting_started/keypoints_augmentation/
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['bboxes_labels']) # Bboxes should have labels, read more at https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/
    )

In [5]:
def train_test_split(src_dir):
    dst_dir_img = src_dir + "images"
    dst_dir_anno = src_dir + "annotations"
    
    if os.path.exists(dst_dir_img) and os.path.exists(dst_dir_anno):
        print("folders exist")
    else:
        os.mkdir(dst_dir_img)
        os.mkdir(dst_dir_anno)
        
    for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
        shutil.copy(jpgfile, dst_dir_img)

    for jsonfile in glob.iglob(os.path.join(src_dir, "*.json")):
        shutil.copy(jsonfile, dst_dir_anno)
        
    output = parent_path + "split_folder_output" + "-" + path.year + "-" + path.month + "-" + path.day 
    
    splitfolders.ratio(src_dir, # The location of dataset
                   output=output, # The output location
                   seed=42, # The number of seed
                   ratio=(.7, .2, .1), # The ratio of split dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )
    
    shutil.rmtree(dst_dir_img)
    shutil.rmtree(dst_dir_anno)
    
    return output  
    

In [6]:
class KPDataset(Dataset):
    def __init__(self, root, transform=None, demo=False):                
        self.root = root
        self.transform = transform
        self.demo = demo # Use demo=True if you need transformed and original images (for example, for visualization purposes)
        self.imgs_files = sorted(os.listdir(os.path.join(root, "images")))
        self.annotations_files = sorted(os.listdir(os.path.join(root, "annotations")))
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "images", self.imgs_files[idx])
        annotations_path = os.path.join(self.root, "annotations", self.annotations_files[idx])

        img_original = cv2.imread(img_path)
        img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
        
        with open(annotations_path) as f:
            data = json.load(f)
            bboxes_original = data['bboxes']
            keypoints_original = data['keypoints']
            
            # All objects are keypoints on the robot
            bboxes_labels_original = [] 
            bboxes_labels_original.append('base_joint')
            bboxes_labels_original.append('joint2')
            bboxes_labels_original.append('joint3')
            bboxes_labels_original.append('joint4')
            bboxes_labels_original.append('joint5')
            bboxes_labels_original.append('joint6')  

        if self.transform:   
            # Converting keypoints from [x,y,visibility]-format to [x, y]-format + Flattening nested list of keypoints            
            # For example, if we have the following list of keypoints for three objects (each object has two keypoints):
            # [[obj1_kp1, obj1_kp2], [obj2_kp1, obj2_kp2], [obj3_kp1, obj3_kp2]], where each keypoint is in [x, y]-format            
            # Then we need to convert it to the following list:
            # [obj1_kp1, obj1_kp2, obj2_kp1, obj2_kp2, obj3_kp1, obj3_kp2]
            keypoints_original_flattened = [el[0:2] for kp in keypoints_original for el in kp]
            
            # Apply augmentations
            transformed = self.transform(image=img_original, bboxes=bboxes_original, bboxes_labels=bboxes_labels_original, keypoints=keypoints_original_flattened)
            img = transformed['image']
            bboxes = transformed['bboxes']
            # Unflattening list transformed['keypoints']
            # For example, if we have the following list of keypoints for three objects (each object has two keypoints):
            # [obj1_kp1, obj1_kp2, obj2_kp1, obj2_kp2, obj3_kp1, obj3_kp2], where each keypoint is in [x, y]-format
            # Then we need to convert it to the following list:
            # [[obj1_kp1, obj1_kp2], [obj2_kp1, obj2_kp2], [obj3_kp1, obj3_kp2]]
            keypoints_transformed_unflattened = np.reshape(np.array(transformed['keypoints']), (-1,1,2)).tolist()

            # Converting transformed keypoints from [x, y]-format to [x,y,visibility]-format by appending original visibilities to transformed coordinates of keypoints
            keypoints = []
            for o_idx, obj in enumerate(keypoints_transformed_unflattened):
#                 print("object", obj)
#                 print(" obj index", o_idx)# Iterating over objects
                obj_keypoints = []
                for k_idx, kp in enumerate(obj): # Iterating over keypoints in each object
                    obj_keypoints.append(kp + [keypoints_original[o_idx][k_idx][2]])
                keypoints.append(obj_keypoints)
        
        else:
            img, bboxes, keypoints = img_original, bboxes_original, keypoints_original        
        
        # Convert everything into a torch tensor        
        bboxes = torch.as_tensor(bboxes, dtype=torch.float32)       
        target = {}
        labels = [1, 2, 3, 4, 5, 6]            
        target["boxes"] = bboxes
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64) # all objects are joint positions
        target["image_id"] = torch.tensor([idx])
        target["area"] = (bboxes[:, 3] - bboxes[:, 1]) * (bboxes[:, 2] - bboxes[:, 0])
        target["iscrowd"] = torch.zeros(len(bboxes), dtype=torch.int64)
        target["keypoints"] = torch.as_tensor(keypoints, dtype=torch.float32)
        img = F.to_tensor(img)        
        bboxes_original = torch.as_tensor(bboxes_original, dtype=torch.float32)
        target_original = {}
        target_original["boxes"] = bboxes_original
        target_original["labels"] = torch.as_tensor(labels, dtype=torch.int64) # all objects are glue tubes
        target_original["image_id"] = torch.tensor([idx])
        target_original["area"] = (bboxes_original[:, 3] - bboxes_original[:, 1]) * (bboxes_original[:, 2] - bboxes_original[:, 0])
        target_original["iscrowd"] = torch.zeros(len(bboxes_original), dtype=torch.int64)
        target_original["keypoints"] = torch.as_tensor(keypoints_original, dtype=torch.float32)        
        img_original = F.to_tensor(img_original)

        if self.demo:
            return img, target, img_original, target_original
        else:
            return img, target
    
    def __len__(self):
        return len(self.imgs_files)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as func
import networkx as nx


class GNNEncoder(nn.Module):
    def __init__(self, vertices_dim=3, hidden_dim=128, num_vertices=6):
        super(GNNEncoder, self).__init__()
        self.f_enc = nn.Linear(vertices_dim, hidden_dim)
        self.f_e1 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.f_v = nn.Linear(hidden_dim, hidden_dim)
        self.f_e2 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.num_vertices = num_vertices
        
    def generate_edges(self, vertices):
        vertices_list = vertices.tolist()
        G = nx.Graph()
        for idx, vertex in enumerate(vertices_list):
            G.add_node(idx, x=vertex[0], y=vertex[1], t=vertex[2])  
            if idx < len(vertices_list) - 1:
                G.add_edge(idx, idx + 1)
        # Add the edge between the last and the first vertex to form a cycle
        G.add_edge(len(vertices_list) - 1, 0)
        edges = torch.tensor(list(G.edges), dtype=torch.long).to(vertices.device)
        return edges

#     def generate_edges(self, vertices):
#         vertices_list = vertices.tolist()
#         G = nx.Graph()
#         for idx, vertex in enumerate(vertices_list):
#             G.add_node(idx, x=vertex[0], y=vertex[1], t=vertex[2])  
#             if idx < len(vertices_list) - 1:
#                 G.add_edge(idx, idx + 1)
#         edges = torch.tensor(list(G.edges), dtype=torch.long).to(vertices.device)
#         return edges
    
#     def generate_edges(self, vertices):
#         num_vertices = vertices.shape[0]
#         all_edges = [[i, j] for i in range(num_vertices) for j in range(i+1, num_vertices)]
#         edges = torch.tensor(all_edges, dtype=torch.long).to(vertices.device)
#         return edges

    def forward(self, vertices):
        h1 = self.f_enc(vertices)
        edges = self.generate_edges(vertices)
        print(edges)
        h1_source = h1[edges[:, 0]]
        h1_target = h1[edges[:, 1]]
        h_e1 = self.f_e1(torch.cat((h1_source, h1_target), dim=1))
        h_j_2 = self.f_v(h_e1)
        h2_source = h_j_2[edges[:, 0]]
        h2_target = h_j_2[edges[:, 1]]
        h_e2 = self.f_e2(torch.cat((h2_source, h2_target), dim=1))
        h_e2_prob = torch.sigmoid(h_e2)
        return vertices, h_e2_prob, edges

class GNNDecoder(nn.Module):
    def __init__(self, vertices_dim=3, hidden_dim=128, num_vertices=6):
        super(GNNDecoder, self).__init__()
        self.f_e = nn.Linear(vertices_dim * 2, hidden_dim)  # Concatenate two vertices features
        self.f_h = nn.Linear(hidden_dim, vertices_dim)  # Transform h_ij to the same dimension as vertices
        self.f_v = nn.Linear(vertices_dim, vertices_dim)  # Update vertex feature

    def forward(self, vertices, h_e2_prob, edges):
        h_source = vertices[edges[:, 0]]
        h_target = vertices[edges[:, 1]]
        h = torch.zeros_like(vertices)

        for idx, (i, j) in enumerate(edges):  # Iterate over edges
            h_ij = h_e2_prob[idx] * self.f_e(torch.cat((h_source[idx], h_target[idx]), dim=0))  # Apply edge function
            h_ij_transformed = self.f_h(h_ij)  # Transform h_ij to the same dimension as vertices
            h[j] += h_ij_transformed  # Accumulate edge features to the target vertex

        h_transformed = self.f_v(h.view(-1, vertices.shape[1]))  # Transform h
        h_transformed = h_transformed.view(vertices.shape)  # Reshape back to original shape
        vertices_g = vertices + h_transformed  # Update vertex features

        return vertices_g  # Return vertices_g as the prediction and vertices_g itself as the mean for Gaussian distribution



In [8]:
class TrifocalLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, vertices_pred, vertices_gt):
        loss = (vertices_gt - torch.tensor(vertices_pred)).pow(2).mean()  # Changed from sum() to mean()
        return loss
    
# def cross_entropy_loss_func(edges_prob, edges_gt):
#     edges_gt_expanded = edges_gt.unsqueeze(-1).float()
#     loss_func = nn.BCEWithLogitsLoss()
#     loss = loss_func(edges_prob, edges_gt_expanded)
#     return loss

def edge_loss(edges_prob, edges_gt):
    # Expand edges_gt to match the shape of edges_prob
    print(edges_prob.shape)
    edges_gt_expanded = torch.zeros(edges_prob.shape, dtype=torch.float32)
    
    for i in range(edges_gt.shape[0]):
        u, v = edges_gt[i]
        print("u:", u)
        print("v:", v)
        print("edges_gt_expanded.shape:", edges_gt_expanded.shape)
        if i < edges_gt_expanded.shape[0]:
            if u < edges_gt_expanded.shape[1]:
                edges_gt_expanded[i, u] = 1
            if v < edges_gt_expanded.shape[1]:
                edges_gt_expanded[i, v] = 1
        print('new u', u)
        print('new v', v)
    # Compute the cross-entropy loss
    loss = -torch.sum(edges_gt_expanded.to(device) * torch.log(torch.clamp(edges_prob, min=1e-7)))
                      
    return loss



In [9]:
class KeypointPipeline(nn.Module):
    def __init__(self):
        super().__init__()
        self.keypoint_model = torch.load(weights_path).to(device)
        self.gnn_encoder = GNNEncoder()
        self.gnn_decoder = GNNDecoder()

    def forward(self, img):
#         print("image in keypoints eval phase", img.shape)
#         img = F.to_tensor(img).to(device)
        img.unsqueeze_(0)
        img = list(img)
        with torch.no_grad():
            self.keypoint_model.to(device)
            self.keypoint_model.eval()
            output = self.keypoint_model(img)
            
        img = (img[0].permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
        scores = output[0]['scores'].detach().cpu().numpy()
        high_scores_idxs = np.where(scores > 0.7)[0].tolist() # Indexes of boxes with scores > 0.7
        post_nms_idxs = torchvision.ops.nms(output[0]['boxes'][high_scores_idxs], \
            output[0]['scores'][high_scores_idxs], 0.3).cpu().numpy() # Indexes of boxes left after applying NMS (iou_threshold=0.3)
        
        keypoints = []
        key_points = []
        for kps in output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            keypoints.append(list(map(int, kps[0,0:2])))
            key_points.append([list(map(int, kp[:2])) for kp in kps])
            
        print("keypoints", keypoints)

        labels = []
        for label in output[0]['labels'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            labels.append(label)
#             labels.append('j' + str(int(label)))
        
        print("labels", labels)
        
        keypoints_ = [x for _,x in sorted(zip(labels,keypoints))]
#         kp_label = [list(x) + [y] for (x, y) in sorted(zip(keypoints, labels))]

        # Create a dictionary where the key is the label and the value is the keypoint
        label_keypoint_dict = {lbl: kp for kp, lbl in zip(keypoints, labels)}

        # Convert the dictionary back to a list and sort it by the label keys
        labeled_keypoints = [value + [key] for key, value in sorted(label_keypoint_dict.items())] #,key=lambda item: int(item[0][1:]))]
        
        print("keypoints_", keypoints_)
        print("kp_label", labeled_keypoints)
                
        keypoints = torch.stack([torch.tensor(kp) for kp in labeled_keypoints]).float().to(device)
        print("Original Keypoints", keypoints)
        vertices, enc_e, edges = self.gnn_encoder(keypoints)
        
        updated_vertices = self.gnn_decoder(vertices, enc_e, edges)
        
        return updated_vertices, enc_e, edges



In [ ]:
# Define the model
model = KeypointPipeline()
model = model.to(device)

# Define the loss
criterion = TrifocalLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Load the trained model
model.keypoint_model = torch.load(weights_path).to(device)

num_epochs = 10  # Define your number of epochs

KEYPOINTS_FOLDER_TRAIN = train_test_split(root_dir) +"/train" #train_test_split(root_dir) +"/train"
KEYPOINTS_FOLDER_VAL = train_test_split(root_dir) +"/val"
KEYPOINTS_FOLDER_TEST = train_test_split(root_dir) +"/test"

dataset_train = KPDataset(KEYPOINTS_FOLDER_TRAIN, transform=train_transform(), demo=False)
dataset_val = KPDataset(KEYPOINTS_FOLDER_VAL, transform=None, demo=False)
dataset_test = KPDataset(KEYPOINTS_FOLDER_TEST, transform=None, demo=False)

data_loader_train = DataLoader(dataset_train, batch_size=1, shuffle=True, collate_fn=collate_fn)
data_loader_val = DataLoader(dataset_val, batch_size=1, shuffle=False, collate_fn=collate_fn)
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate_fn)


model.train()
# Epoch loop
for epoch in range(num_epochs):
    # For each batch in your training data
    for batch in data_loader_train:
        img_tuple, target_dict_tuple = batch
        img = img_tuple[0]
#         print(img.shape)
        target = target_dict_tuple[0]
        img = img.to(device)
        vertices_gt = target['keypoints'].to(device)
        num_vertices = vertices_gt.shape[0]
        print(num_vertices)
        vertices_gt[:, :, 2] = torch.arange(1, num_vertices+1).unsqueeze(1).to(device)

        # Forward pass
        vertices_pred, edges_prob, edges_gt = model(img)
        
        # Compute the losses
        trifocal_loss = criterion(vertices_pred, vertices_gt)
        ce_loss = edge_loss(edges_prob, edges_gt)

        # Combined loss
        loss = trifocal_loss + ce_loss
#         loss = trifocal_loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print loss for each epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Copying files: 2662 files [00:00, 18491.09 files/s]
Copying files: 2662 files [00:00, 17422.05 files/s]
Copying files: 2662 files [00:00, 17139.45 files/s]


6
keypoints [[111, 258], [196, 258], [222, 180], [201, 178], [306, 232], [310, 255]]
labels [1, 2, 4, 3, 5, 6]
keypoints_ [[111, 258], [196, 258], [201, 178], [222, 180], [306, 232], [310, 255]]
kp_label [[111, 258, 1], [196, 258, 2], [201, 178, 3], [222, 180, 4], [306, 232, 5], [310, 255, 6]]
Original Keypoints tensor([[111., 258.,   1.],
        [196., 258.,   2.],
        [201., 178.,   3.],
        [222., 180.,   4.],
        [306., 232.,   5.],
        [310., 255.,   6.]], device='cuda:0')
tensor([[0, 1],
        [0, 5],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5]], device='cuda:0')
torch.Size([6, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(1, device='cuda:0')
u: tensor(0, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(5, device='cuda:0')
u: tensor(1, device='

/tmp/ipykernel_96563/4190947107.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = (vertices_gt - torch.tensor(vertices_pred)).pow(2).mean()  # Changed from sum() to mean()


keypoints [[381, 112], [381, 196], [386, 278], [405, 271], [321, 352], [300, 354]]
labels [1, 2, 4, 3, 5, 6]
keypoints_ [[381, 112], [381, 196], [405, 271], [386, 278], [321, 352], [300, 354]]
kp_label [[381, 112, 1], [381, 196, 2], [405, 271, 3], [386, 278, 4], [321, 352, 5], [300, 354, 6]]
Original Keypoints tensor([[381., 112.,   1.],
        [381., 196.,   2.],
        [405., 271.,   3.],
        [386., 278.,   4.],
        [321., 352.,   5.],
        [300., 354.,   6.]], device='cuda:0')
tensor([[0, 1],
        [0, 5],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5]], device='cuda:0')
torch.Size([6, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(1, device='cuda:0')
u: tensor(0, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(5, device='cuda:0')
u: tensor(1, device='cu

keypoints [[258, 283], [257, 366], [303, 214], [283, 207], [400, 198], [418, 208]]
labels [2, 1, 4, 3, 5, 6]
keypoints_ [[257, 366], [258, 283], [283, 207], [303, 214], [400, 198], [418, 208]]
kp_label [[257, 366, 1], [258, 283, 2], [283, 207, 3], [303, 214, 4], [400, 198, 5], [418, 208, 6]]
Original Keypoints tensor([[257., 366.,   1.],
        [258., 283.,   2.],
        [283., 207.,   3.],
        [303., 214.,   4.],
        [400., 198.,   5.],
        [418., 208.,   6.]], device='cuda:0')
tensor([[0, 1],
        [0, 5],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5]], device='cuda:0')
torch.Size([6, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(1, device='cuda:0')
u: tensor(0, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(5, device='cuda:0')
u: tensor(1, device='cu

...

keypoints [[257, 366], [258, 283], [278, 203], [257, 203], [373, 229], [391, 241]]
labels [1, 2, 4, 3, 5, 6]
keypoints_ [[257, 366], [258, 283], [257, 203], [278, 203], [373, 229], [391, 241]]
kp_label [[257, 366, 1], [258, 283, 2], [257, 203, 3], [278, 203, 4], [373, 229, 5], [391, 241, 6]]
Original Keypoints tensor([[257., 366.,   1.],
        [258., 283.,   2.],
        [257., 203.,   3.],
        [278., 203.,   4.],
        [373., 229.,   5.],
        [391., 241.,   6.]], device='cuda:0')
tensor([[0, 1],
        [0, 5],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5]], device='cuda:0')
torch.Size([6, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(1, device='cuda:0')
u: tensor(0, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(5, device='cuda:0')
u: tensor(1, device='cu

keypoints [[381, 112], [380, 195], [328, 255], [312, 241], [217, 216], [217, 199]]
labels [1, 2, 3, 4, 5, 6]
keypoints_ [[381, 112], [380, 195], [328, 255], [312, 241], [217, 216], [217, 199]]
kp_label [[381, 112, 1], [380, 195, 2], [328, 255, 3], [312, 241, 4], [217, 216, 5], [217, 199, 6]]
Original Keypoints tensor([[381., 112.,   1.],
        [380., 195.,   2.],
        [328., 255.,   3.],
        [312., 241.,   4.],
        [217., 216.,   5.],
        [217., 199.,   6.]], device='cuda:0')
tensor([[0, 1],
        [0, 5],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5]], device='cuda:0')
torch.Size([6, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(1, device='cuda:0')
u: tensor(0, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(5, device='cuda:0')
u: tensor(1, device='cu

keypoints [[381, 112], [380, 195], [431, 261], [444, 244], [334, 247], [311, 245]]
labels [1, 2, 4, 3, 5, 6]
keypoints_ [[381, 112], [380, 195], [444, 244], [431, 261], [334, 247], [311, 245]]
kp_label [[381, 112, 1], [380, 195, 2], [444, 244, 3], [431, 261, 4], [334, 247, 5], [311, 245, 6]]
Original Keypoints tensor([[381., 112.,   1.],
        [380., 195.,   2.],
        [444., 244.,   3.],
        [431., 261.,   4.],
        [334., 247.,   5.],
        [311., 245.,   6.]], device='cuda:0')
tensor([[0, 1],
        [0, 5],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5]], device='cuda:0')
torch.Size([6, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(1, device='cuda:0')
u: tensor(0, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(5, device='cuda:0')
u: tensor(1, device='cu

keypoints [[258, 367], [301, 213], [258, 283], [398, 249], [281, 207], [385, 265]]
labels [1, 4, 2, 6, 3, 5]
keypoints_ [[258, 367], [258, 283], [281, 207], [301, 213], [385, 265], [398, 249]]
kp_label [[258, 367, 1], [258, 283, 2], [281, 207, 3], [301, 213, 4], [385, 265, 5], [398, 249, 6]]
Original Keypoints tensor([[258., 367.,   1.],
        [258., 283.,   2.],
        [281., 207.,   3.],
        [301., 213.,   4.],
        [385., 265.,   5.],
        [398., 249.,   6.]], device='cuda:0')
tensor([[0, 1],
        [0, 5],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5]], device='cuda:0')
torch.Size([6, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(1, device='cuda:0')
u: tensor(0, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(5, device='cuda:0')
u: tensor(1, device='cu

keypoints [[112, 258], [196, 258], [244, 191], [225, 184], [337, 225], [337, 247]]
labels [1, 2, 4, 3, 5, 6]
keypoints_ [[112, 258], [196, 258], [225, 184], [244, 191], [337, 225], [337, 247]]
kp_label [[112, 258, 1], [196, 258, 2], [225, 184, 3], [244, 191, 4], [337, 225, 5], [337, 247, 6]]
Original Keypoints tensor([[112., 258.,   1.],
        [196., 258.,   2.],
        [225., 184.,   3.],
        [244., 191.,   4.],
        [337., 225.,   5.],
        [337., 247.,   6.]], device='cuda:0')
tensor([[0, 1],
        [0, 5],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5]], device='cuda:0')
torch.Size([6, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(1, device='cuda:0')
u: tensor(0, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([6, 128])
new u tensor(0, device='cuda:0')
new v tensor(5, device='cuda:0')
u: tensor(1, device='cu